# Making the distributions of most massive galaxies

This notebook is an example of how to get the cumulative density function (CDF) of the posterior, sampling individual galaxy counts from a gamma distribution as in Steinhardt, Jespersen & Linzer (https://ui.adsabs.harvard.edu/abs/2021ApJ...923....8S/abstract).

The notebook goes through the sampling process and how to supersample the bins to get a posterior invariant to the number of subsamples

Examples of plotting are done in the companion notebook make_figures.ipynb

The cosmic variance is calculated as in get_cosmic_variance.ipynb with the ``` Python ``` version of the 2011 Moster, Somerville et al. ```IDL``` Cosmic Variance Cookbook (https://arxiv.org/abs/1001.1737).

The ``` Python ``` version was written for this project, since very few people use, let alone have licenses for, ``` IDL ``` these days, so hopefully more people can do their own cosmic variance calculations for the high redshift limit where it is so important!


In [1]:
import hmf
from hmf import functional
import pandas as pd

import numpy as np
import scipy.integrate
import astropy
from astropy.cosmology import LambdaCDM, Planck18_arXiv_v2
from scipy.stats import norm
from scipy.ndimage import gaussian_filter1d, gaussian_filter
import pickle, time

In [2]:
cosmo = Planck18_arXiv_v2 # Define cosmology for volume calculation
tot_sky = 41253. # Total square degrees in the sky

halo_model = "SMT" # Select 

mmin = 3. # Minimum halo mass for HMF
mmax = 14.5 # Maximum halo mass for HMF

baryon_frac = 0.0224/0.120 # from Planck18

minMass = 3. # Minimum stellar mass
maxMass = 12.5 # Maximum stellar mass
dM = 0.5 # Bin sizes
Nm = 100 # Subdivisions of those bins

mbins = np.arange(minMass, maxMass+dM/Nm-1e-10, dM/Nm)

save = True #whether to save or not

corr = .25 #corrections for raw M_max from supersampling 0.5 dex bin
mbins=np.round(mbins, 3) #numpy likes decimals, I do not

In [3]:
# Method to take N trials of a gamma distribution with a given variance and mean

# Sig_v should just be cosmic variance, the poisson variance is added in the method

def trial(sig_v, mean, Ntrials=10000):
    ''' Method taking N trials of a gamma distribution with some mean galaxy number and cosmic variance
    Inputs:
    sig_v: Cosmic variance, given as a fraction so that sigma_cv,field = mean * sigma_cv
    mean: The mean of the distribution
    Ntrials: Number of samples
    
    returns: Samples from gamma distribution'''
    
    Ntrials = int(Ntrials)
    var = sig_v**2*mean**2+mean # total variance, cv + poisson, see https://arxiv.org/abs/1001.1737 for definition
    k = mean**2/var
    t = var/mean
    y = np.linspace(1e-6,1-1e-6, Ntrials)
    rand = np.rint(t*scipy.special.gammaincinv(k, y, out = None))
    return rand

In [ ]:
Ntrials = int(1e4) #turn up to 1e5 - 1e6 for more accurate results and always investigate the posteriors. 
# If the posteriors look weird, turn this up even more. Fewer trials are needed for lower sigma boundaries
# In general, the larger the survey, the more trials needed, since the posterior boundaries are very tightly spaced

surveys = ['single', 'ceers'] #here for a single JSWT pointing and for a CEERS - like survey
areas = [2*4.84, 7*4.84] #in arcminutes 


for survey, area in zip(surveys, areas): 

    print(f'Calculating posteriors for {survey} survey')
    survey_area = area/3600 # Survey area in square degrees
    cv_df = pd.read_csv(f'dfs/{survey}.csv', sep =',') # get cosmic variance DataFrame
    
    
    # force cosmic variance to be monotonic, since small numerical issues in Moster's calculator makes this a non-given
    # for very high z
    
    cv_df['10.0'][cv_df['9.5']/cv_df['10.0']>1] = cv_df['9.5'][cv_df['9.5']/cv_df['10.0']>1]

    ## Following Moster+ 11, set everything below the limit to have the same cosmic variance
    ## It's probably wrong but it's the best we know of
    cv_df['6.5'] = cv_df['7.0']
    cv_df['6.0'] = cv_df['7.0']
    cv_df['5.5'] = cv_df['7.0']
    cv_df['5.0'] = cv_df['7.0']
    cv_df['4.5'] = cv_df['7.0']

    ## Linear extrapolation to higher masses
    ## This is never relevant for JSWT but is included in the case someone wants to try out an all-sky survey
    cv_df['11.5'] = cv_df['11.0']**2/cv_df['10.5']
    cv_df['12.0'] = cv_df['11.5']**2/cv_df['11.0']
    cv_df['12.5'] = cv_df['12.0']**2/cv_df['11.5']

    ## interpolating cosmic variance values and smoothing the curves
    ms = [float(m) for m in cv_df.columns[3:]]

    spline_zs = [] 

    for i in range(len(cv_df)):
        cv_sort = cv_df[cv_df.columns[3:]].iloc[i]

        cv_sort = [x for (y, x) in sorted(zip(ms, cv_sort))]

        ip0 = scipy.interpolate.UnivariateSpline(np.sort(ms), cv_sort)
        spline_z = ip0(mbins)

        mincv = np.min(cv_sort)
        spline_z = np.where(spline_z>mincv, spline_z, mincv)

        spline_z = np.where(mbins<7.0,  mincv, spline_z) # keep everything below 10^7 the same

        spline_zs.append(spline_z)

    cols = [str(m) for m in mbins]

    #re-enforce monotonicity
    cvss = np.maximum.accumulate(np.vstack(spline_zs), axis=0)
    cvss = np.maximum.accumulate(cvss, axis=1)

    # smooth values so that no bin-edges will do funky stuff
    # the sigma used here can only be used for the same Nm and bin-size as in the original paper
   
    cvss = gaussian_filter(np.vstack(cvss), sigma = [1,25])

    #final cosmic variance data frame
    cv_df[cols] = cvss 

    dlog10m = 0.001 #numerical HMF resolution, should be << desired mass resolution
    Ndm = int(0.5//dlog10m) #number of steps in a 0.5 dex mass interval for HMF

    zs = cv_df['z'] # get redshift array
    dz = 1 # redshift window width

    Ms = [] 
    cdfs = []
    pdfnonorm = []
    meansN = []

    Nss = []
    cvss = []

    for z in zs:
        start = time.time()
        # could possibly include some uncertainty here in the future, as in Lovell+23
        # however, this would have to be propagated for different samples later on, so left for future work
        sbf = 0.05 + 0.024 * (z - 4)
        
        # get volume in comoving coordinates
        vol = (cosmo.comoving_volume(z + dz/2).value - cosmo.comoving_volume(z - dz/2).value) * (survey_area/tot_sky)
        
        trials = []
        
        # get halo mass function for given halo model, redshift and numerical boundaries
        for quants, h, l in functional.get_hmf(['dndm','m'],z = z,Mmin = mmin, Mmax = mmax,\
                                               hmf_model = halo_model, dlog10m = dlog10m):

            dNdm = h.dndm*vol # get dNdm, the halo mass function in the given volume
            mass = h.m # get mass range

            little_h = h.cosmo_model.h # get little h, h = H / 100 km/s/Mpc
            mass = mass * little_h # cast mass to actual masses

            dNdm = dNdm / little_h**4 # go away from comoving dNdm (1 h from mass, h^3 from volume)

            stellar_mass = mass * sbf * baryon_frac # cast halo mass to stellar mass

            N_trapz = [] # get total mass function, N in [M-0.25:M+0.25].
            for i in range(len(mass)-Ndm):
                inte = np.trapz(dNdm[i:i+Ndm], mass[i:i+Ndm]) #integrate over bin
                N_trapz.append(inte)
            N_trapz = np.array(N_trapz)   

            stellar_mass = stellar_mass[int(Ndm)//2:-int(Ndm)//2] #redefine to fit integration range

            print(z, sbf)
            
            Ns = []
            cvs = []
            meansN.append(np.log10(stellar_mass)[np.argmin(np.abs(N_trapz-1))-1])
            for m in mbins[:-1]:
                arg = np.argmin(np.abs(np.log10(stellar_mass)-m))
                N = N_trapz[arg-1]
                Ns.append(N)
                cv = cv_df[cv_df['z']==z][str(m)]
                cvs.append(float(cv))
                trials.append(trial(float(cv), N, Ntrials = Ntrials))
            
            # save
            Nss.append(Ns)   
            cvss.append(cvs)
            
            mgrid = np.tile(mbins,(len(trials[0]), 1)).T
            M = np.max(np.where(np.vstack(trials)>0, mgrid[:-1], -np.inf), axis=0)
            M = np.nan_to_num(M, neginf = np.nan)
            Ms.append(M-corr) #correction to raw pdf to account for oversampling when visualizing


            # make cdf from super-sampled bins
            # Do not normalize everything together, each supersampling needs to be normalized by itself
            # which will be done later
            pdfnn, _, = np.histogram(M, bins = list(mbins-dM/Nm)+[max(mbins)+dM/Nm], density=0)

            pdfnonorm.append(pdfnn) #save for later
            cdfs_in = []
            ms_in = []

            for i in range(Nm):
                cdfs_in.append(np.cumsum(pdfnn[i::Nm]/np.sum(pdfnn[i::Nm]))) # take individual supersamples in bins
                ms_in.append(mbins[i::Nm]) # get masses corresponding to supersampling

            mnew, cdfnew = np.hstack(ms_in), np.hstack(cdfs_in)
            a = np.vstack([mnew, cdfnew]).T
            _, cdf = a[a[:, 0].argsort()].T # sort by mass

            cdf = gaussian_filter1d(cdf, 5) #needed to get smooth pdfs, very small diffusion length scale

            cdfs.append(cdf)

        print('Time', np.round(time.time()-start, 2))

    if save:
        with open(f'posteriors/{survey}_Ms_{int(np.log10(Ntrials))}.pkl', 'wb') as handle:
            pickle.dump(Ms, handle)
        with open(f'posteriors/{survey}_cdfs_{int(np.log10(Ntrials))}.pkl', 'wb') as handle:
            pickle.dump(cdfs, handle)
        with open(f'posteriors/{survey}_cvss_{int(np.log10(Ntrials))}.pkl', 'wb') as handle:
            pickle.dump(cvss, handle)

Calculating posteriors for single survey
5.5 0.08600000000000001
Time 116.83
6.0 0.098
Time 111.92
6.5 0.11
Time 110.15
7.0 0.12200000000000001
Time 116.23
7.5 0.134
Time 122.05
8.0 0.14600000000000002
Time 125.22
8.5 0.158
Time 131.41
9.0 0.16999999999999998
Time 149.28
9.5 0.182
Time 158.99
10.0 0.194
Time 147.66
10.5 0.20600000000000002
Time 133.99
11.0 0.21800000000000003
Time 127.5
11.5 0.22999999999999998
Time 126.19
12.0 0.242
Time 138.4
12.5 0.254
Time 148.69
13.0 0.266
Time 157.42
13.5 0.278
Time 161.34
14.0 0.29
Time 155.1
14.5 0.302
Time 155.16
15.0 0.314
Time 158.06
15.5 0.326
Time 159.05
16.0 0.338
Time 160.02
16.5 0.35
Time 156.48
17.0 0.362
Time 173.58
17.5 0.374
Time 172.93
18.0 0.386
Time 152.94
18.5 0.398
Time 143.54
19.0 0.41
Time 139.5
19.5 0.422
Time 139.84
Calculating posteriors for ceers survey
5.5 0.08600000000000001
Time 107.78
6.0 0.098
Time 104.73
6.5 0.11
Time 104.75
7.0 0.12200000000000001
Time 105.72
7.5 0.134
